In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import pennylane as qml
import warnings
warnings.filterwarnings('ignore')
tf.keras.backend.set_floatx('float64')

import warnings
warnings.filterwarnings('ignore')

In [ ]:
###predict 2nd half line using 1st half total and open ##

df1 = pd.read_csv("nfl_odds.csv")
df1['1H'] = df1['1st'] + df1['2nd']

df2 = pd.read_csv('bet.csv')
df = df1.merge(df2, left_on = 'Team', right_on = 'Tm')
df.columns

In [ ]:
df = df[['1H','Open', 'Sc%','EXP', 'Y/P', 'TD', 'TO%','PF','Yds','ML', '2H']]
df.head()

In [ ]:
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def qnode(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubits)]



In [ ]:
n_layers = 6
weight_shapes = {"weights": (n_layers, n_qubits)}

In [ ]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubits)

In [ ]:
clayer_1 = tf.keras.layers.Dense(4)
clayer_2 = tf.keras.layers.Dense(2, activation="relu")
model = tf.keras.models.Sequential([clayer_1, qlayer, clayer_2])

In [ ]:
opt = tf.keras.optimizers.SGD(learning_rate=0.2)
model.compile(opt, loss="mae", metrics=["mean_absolute_error"])

In [ ]:
df = df[df.Open != 'pk']
df = df[df['2H'] != 'pk']
df['Open'] = df['Open'].astype(float)
df['2H'] = df['2H'].astype(float)
X = df[['1H','Open', 'Sc%','EXP', 'Y/P', 'TD', 'TO%','PF','Yds','ML']]
y = df['2H']

X = np.asarray(X).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X_train)
X_train = scaler.transform(X_train)

In [ ]:
fitting = model.fit(X_train, y_train, epochs=10, batch_size=5, validation_split=0.15, verbose=2)

In [ ]:
X_test = scaler.transform(X_test)
preds = model.predict(X_test)

In [ ]:
pred = pd.DataFrame(preds, columns =[ 'prediction'])
pred = pred[(pred.prediction > 0) & (pred.prediction < 30)]
y_test = y_test.reset_index()

compare = pd.concat([pred, y_test], axis=1)
compare = compare.drop('index', axis=1)

In [ ]:
compare[(compare.prediction - compare['2H']) < 8]